In [2]:
def calculator(expr: str):
    return eval(expr)

def order_status(order_id: int):
    return f"Order {order_id} is shipped"


In [3]:
from typing import TypedDict

class AgentState(TypedDict):
    input: str
    reasoning: str
    action: str
    observation: str
    output: str


In [4]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")

def reasoning_node(state: AgentState):
    prompt = f"""
You are a ReAct agent.

User query: {state['input']}

Decide:
- reasoning
- action (calculator / order_status / none)
Return in this format:
REASONING:
ACTION:
"""
    response = llm.invoke(prompt)
    text = response.content

    reasoning, action = text.split("ACTION:")
    return {
        "reasoning": reasoning.replace("REASONING:", "").strip(),
        "action": action.strip()
    }


C:\Users\apoliset\AppData\Local\Temp\ipykernel_30876\3381925528.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


In [5]:
def action_node(state: AgentState):
    action = state["action"]

    if action.startswith("calculator"):
        expr = action.replace("calculator", "").strip()
        return {"observation": str(calculator(expr))}

    if action.startswith("order_status"):
        order_id = int(action.replace("order_status", "").strip())
        return {"observation": order_status(order_id)}

    return {"observation": "No action taken"}


In [6]:
def final_node(state: AgentState):
    prompt = f"""
User question: {state['input']}
Reasoning: {state['reasoning']}
Observation: {state['observation']}

Provide final answer.
"""
    response = llm.invoke(prompt)
    return {"output": response.content}


In [7]:
from langgraph.graph import StateGraph

graph = StateGraph(AgentState)

graph.add_node("reasoning", reasoning_node)
graph.add_node("action", action_node)
graph.add_node("final", final_node)

graph.set_entry_point("reasoning")
graph.add_edge("reasoning", "action")
graph.add_edge("action", "final")

react_agent = graph.compile()


In [8]:
react_agent.invoke({
    "input": "What is 20 * 5 and check order status for order 101"
})


SyntaxError: invalid syntax (<string>, line 1)